In [1]:
import pandas as pd
import numpy as np
import torch
from importlib import reload
from tqdm.notebook import tqdm
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from transformers import GPT2Config, GPT2Tokenizer

pl.seed_everything(42)

42

## config and tokenizer

In [15]:
import yaml

config = yaml.load(open('configs/config.yaml'))
config

<ipython-input-15-d8f47aef2a1e>:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open('configs/config.yaml'))


{'data': {'batch_size': 10,
  'max_len': 256,
  'csv_file': 'data/processed.csv',
  'tokenizer_name': 'microsoft/DialoGPT-small',
  'val_frac': 0.1},
 'model': {'n_positions': 256,
  'n_ctx': 256,
  'n_embd': 512,
  'n_layer': 4,
  'n_head': 8,
  'n_inner': 768},
 'opt': {'lr': 5e-05, 'max_epochs': 100, 'weight_decay': 0.001}}

In [25]:
tokenizer = GPT2Tokenizer.from_pretrained(config['data']['tokenizer_name'])
tokenizer.pad_token = tokenizer.eos_token

In [16]:
config['model']['vocab_size'] = tokenizer.vocab_size
config['model']['pad_token_id'] = tokenizer.pad_token_id
config['model']['eos_token_id'] = tokenizer.eos_token_id
config['model']['bos_token_id'] = tokenizer.bos_token_id
config

{'data': {'batch_size': 10,
  'max_len': 256,
  'csv_file': 'data/processed.csv',
  'tokenizer_name': 'microsoft/DialoGPT-small',
  'val_frac': 0.1},
 'model': {'n_positions': 256,
  'n_ctx': 256,
  'n_embd': 512,
  'n_layer': 4,
  'n_head': 8,
  'n_inner': 768,
  'vocab_size': 50257,
  'pad_token_id': 50256,
  'eos_token_id': 50256,
  'bos_token_id': 50256},
 'opt': {'lr': 5e-05, 'max_epochs': 100, 'weight_decay': 0.001}}

## data

In [3]:
df = pd.read_csv(config['data']['csv_file'])
with open('text.txt','w') as writer:
    writer.write('\n'.join(df['text']))

In [4]:
contexted = []
n = 7
for i in tqdm(range(n, len(df['text']))):
    row = []
    prev = i - 1 - n # we additionally substract 1, so row will contain current responce and 7 previous responces  
    for j in range(i, prev, -1):
        row.append(df['text'][j])
    contexted.append(row)  
    
columns = ['response', 'context'] 
columns = columns + ['context/'+str(i) for i in range(n-1)]
df = pd.DataFrame.from_records(contexted, columns=columns)

In [21]:
import src.dataset
reload(src.dataset)
from src.dataset import get_dataloaders

train_loader, val_loader = get_dataloaders(tokenizer, df, 
                                           max_len=config['data']['max_len'], 
                                           batch_size=config['data']['batch_size'], 
                                           val_frac=config['data']['val_frac'])

train dataset has 93291 samples and val dataset has 10365 samples


In [8]:
for batch in train_loader:
    for k,v in batch.items():
        print(k,v.shape)
    break

input_ids torch.Size([32, 256])
attention_mask torch.Size([32, 256])


## model

In [17]:
import src.model
reload(src.model)
from src.model import ErfBot


model = ErfBot(config=GPT2Config(**config['model']), 
               **config['opt'])
model

ErfBot(
  (model): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 512)
      (wpe): Embedding(256, 512)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): Block(
          (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): Block(
          (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resi

In [18]:
model.count_parameters()

33225216

## trainer

In [19]:
logger = TensorBoardLogger(
    save_dir='logs/',
    name='gpt2_logs'
)

checkpoint = ModelCheckpoint(dirpath='weights/gpt2', 
                             filename='{epoch}-{val_loss:.2f}', 
                             monitor='val_loss',
                             save_top_k=1, 
                             period=1)

lr_logger = LearningRateMonitor(logging_interval='step')

## defining trainer
trainer = pl.Trainer(benchmark=True, 
                  gpus=1, 
                  logger=logger, 
                  max_epochs=config['opt']['max_epochs'],
                  callbacks=[checkpoint, lr_logger])

GPU available: True, used: True
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [22]:
trainer.fit(model, train_loader, val_loader)


  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 33.2 M
------------------------------------------
33.2 M    Trainable params
0         Non-trainable params
33.2 M    Total params

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 33.2 M
------------------------------------------
33.2 M    Trainable params
0         Non-trainable params
33.2 M    Total params


RuntimeError: CUDA out of memory. Tried to allocate 1.53 GiB (GPU 0; 5.80 GiB total capacity; 3.33 GiB already allocated; 128.62 MiB free; 4.83 GiB reserved in total by PyTorch)